# Music Generation using Meta's MusicGen 

### Library imports

In [1]:
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write
import torchaudio
import gradio as gr
import warnings
warnings.filterwarnings('ignore')

### Model imports

In [2]:
model = MusicGen.get_pretrained('melody')

In [3]:
model.set_generation_params(
    use_sampling=True,
    top_k=250,
    duration=5
)

### Text-conditional Generation

In [4]:
def genfromtext(description, duration):
    model.set_generation_params(use_sampling=True, duration=duration)   
    audio_values = model.generate(descriptions= [description])
    return (32000, audio_values.cpu().detach().numpy())

### Melody-conditional Generation

In [5]:
def genfromaudio(melody, description, duration):
    model.set_generation_params(use_sampling=True, duration=duration)
    melody_waveform, sr = torchaudio.load(melody)
    melody_waveform = melody_waveform.unsqueeze(0).repeat(1, 1, 1)
    audio_values = model.generate_with_chroma(descriptions=[description],melody_wavs=melody_waveform, melody_sample_rate=sr,progress=True)
    return (32000, audio_values.cpu().detach().numpy())

### Gradio Interface

In [6]:
interface1 = gr.Interface(
    genfromtext, inputs=[
        gr.Text(label="Input Text", interactive=True),
        gr.Number(minimum=1, maximum=10, value=5, label="Duration", interactive=True)
    ], outputs=[
        gr.Audio(type="numpy",label="Generated Music")
    ],
    description="Text-Conditional Generation")

In [7]:
interface2 = gr.Interface(
    genfromaudio, inputs= [
        gr.Audio(source="upload", type="filepath", label="Melody (Upload an audio with a max duration of 30sec)", interactive=True),
        gr.Text(label="Input Text", interactive=True),
        gr.Number(minimum=1, maximum=10, value=5, label="Duration", interactive=True)
    ], outputs=[
        gr.Audio(type="numpy", label="Generated Music")
    ],
    description="Melody-Conditional Generation")

In [8]:
title = "Music Generation using Meta's MusicGen"
final_interface = gr.TabbedInterface([interface1, interface2], ["Text-Conditional", "Melody-Conditional (Upload mp3 audio not exceeding 30sec)"], title=title,theme=gr.themes.Default())
final_interface.launch(inbrowser=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
